In [8]:
import numpy as np
import pandas as pd
import gc
import pickle
import os
import datetime as dt

#plot
import matplotlib.pyplot as plt

#LightGBM
import lightgbm as lgb

#sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

#表示桁数の指定
pd.options.display.float_format = '{:.4f}'.format

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [9]:
# --- scripts パス追加 ---
import sys
if "../scripts" not in sys.path:
    sys.path.append("../scripts")

# --- utils ---
import utils
import importlib
importlib.reload(utils)
from utils import reduce_mem_usage

# --- basic_feature ---
import basic_feature
importlib.reload(basic_feature)
from basic_feature import preprocess_features

# --- train_runner ---
import train_runner
importlib.reload(train_runner)
from train_runner import run_baseline_training

# --- train ---
import train
importlib.reload(train)
from train import train_lgb_regression


In [10]:
# 学習データ読み込み、メモリ削減、オブジェクトに変換
df = pd.read_csv("../data/train.csv")
df = reduce_mem_usage(df)
df = preprocess_features(df)

Memory usage decreased from 68.66 MB to 44.35 MB


In [11]:
#特徴量エンジニアリング1
# 欠損フラグを追加（1: 欠損あり、0: 欠損なし）
# 補完前のオリジナル列（欠損あり）をコピー
df['Episode_Length_minutes_raw'] = df['Episode_Length_minutes']
df['Episode_Length_minutes_was_missing'] = df['Episode_Length_minutes'].isna().astype(int)
# 欠損を中央値で補完
df['Episode_Length_minutes'] = df['Episode_Length_minutes'].fillna(df['Episode_Length_minutes'].median())

#上記と同様の操作を行う
df['Guest_Popularity_percentage_raw'] = df['Guest_Popularity_percentage']
df['Guest_Popularity_percetage_was_missing'] = df['Guest_Popularity_percentage'].isna().astype(int)
df['Guest_Popularity_percentage'] = df['Guest_Popularity_percentage'].fillna(df['Guest_Popularity_percentage'].median())




In [12]:
#データセット作成
x_train = df.drop(["Listening_Time_minutes", "id",], axis=1)
y_train = df["Listening_Time_minutes"]
id_train = df[["id"]]

In [ ]:
x_train.shape
x_train.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Length_minutes_raw,Episode_Length_minutes_was_missing,Guest_Popularity_percentage_raw,Guest_Popularity_percetage_was_missing
0,Mystery Matters,Episode 98,63.8438,True Crime,74.8125,Thursday,Night,53.5938,0.0000,Positive,NaN,1,NaN,1
1,Joke Junction,Episode 26,119.8125,Comedy,66.9375,Saturday,Afternoon,75.9375,2.0000,Negative,119.8125,0,75.9375,0
2,Study Sessions,Episode 16,73.8750,Education,70.0000,Tuesday,Evening,8.9688,0.0000,Negative,73.8750,0,8.9688,0
3,Digital Digest,Episode 45,67.1875,Technology,57.2188,Monday,Morning,78.6875,2.0000,Positive,67.1875,0,78.6875,0
4,Mind & Body,Episode 86,110.5000,Health,80.0625,Monday,Afternoon,58.6875,3.0000,Neutral,110.5000,0,58.6875,0


In [13]:
train_oof, imp, metrics = run_baseline_training(x_train, y_train, id_train)

-------------------- Fold 0 --------------------
Train shape: (600000, 14), Validation shape: (150000, 14)
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[100]	training's rmse: 12.9785	valid_1's rmse: 13.0301
[200]	training's rmse: 12.824	valid_1's rmse: 12.9915
[300]	training's rmse: 12.7203	valid_1's rmse: 12.9757
[400]	training's rmse: 12.6299	valid_1's rmse: 12.9646
[500]	training's rmse: 12.5482	valid_1's rmse: 12.9597
[600]	training's rmse: 12.472	valid_1's rmse: 12.9554
[700]	training's rmse: 12.3996	valid_1's rmse: 12.949
[800]	training's rmse: 12